# Resolução de puzzles Sokoban

## Nota sobre este relatório

Foi utilizada a ferramenta Jupyter Notebook, para tirar proveito de algumas funcionalidades do IPython para análise estatística da execução dos algoritmos.

## Formulação

Foi escolhido representar um estado do puzzle sokoban usando uma classe, `EstadoSokoban`, que vai ser descrita nesta capítulo.

### Atributos

A `EstadoSokoban` contém os seguintes atributos:

- `tabuleiro`: lista de listas que representa o puzzle: Cada uma das listas interiores vão ter:

    - ’#’   – para representar as paredes;
    
    - ’.’   – para representar as posições livres;
    
    - ’*’   – para representar as caixas;
    
    - ’o’   – (um ó minúsculo) para representar os alvos;
    
    - ’A’   – para representar o arrumador.
    
    - ’@’   – para representar uma caixa numa posição alvo.
    
    - ’B’   – para representar o arrumador em cima de uma posição alvo.
    
- `arrumador`: posição do arrumador, tuplo xy.

- `caixas`: posição das caixas, tuplo xy.

- `alvos`: posição dos alvos, tuplo xy.

- `deadlocks`: posição de deadlocks, executando `deadlocks_tabuleiro()`

### Métodos

Para além da representação, foram implementados os seguintes métodos:

- `pos_livre(self, x, y)`: verifica se uma posição do puzzle está livre, isto é, se o arrumador por andar para lá.

- `pos_caixa(self, x, y)`: verifica se uma posição do puzzle tem uma caixa.

- `ver_*(self, x, y)`: usa as funções anteriores para verificar a posição "cima", "baixo", "direita" e "esquerda" (cada uma destas opções são um método diferente.

### Implementação da classe concreta `Problem`

Para aplicar os algoritmos de procura de espaço de estados, foi usada uma implementação concreta da classe `Problem` que para além dos métodos abstratos (`actions()`, `result()`, `path_cost()` e `goal_test()`, foram ainda implementados os seguintes:

- `pos_deadlock_canto`: Deteção de um estado em que uma caixa ficaria num canto.

- `deadlock_parede`: (Paulo get in here ahah)

- `em_cruzamento_com_alvo`: todo

- `deadlocks_tabuleiro`: todo


## Heurísticas definidas

### Distância euclidiana do arrumador ao caixa mais próximo dele

método: `heur_euclidean_usher_target(nodo)`

A primeira heuristica programada foi a distância euclidiana do arrumador à caixa mais próximo. Esta heuristica é útil em puzzles em que o arrumador se encontra muito longe das caixas, ajudando a cortar muitos estados considerados ínuteis.

### Distância euclidiana do arrumador ao alvo mais próximo dele

método: `heur_euclidean_usher_box(nodo)`

A segunda heuristica programada foi a distância euclidiana do arrumador ao alvo mais próximo. Esta heuristica permite também aproximar o arrumador do seu objectivo em puzzles com muitos espaços livres.

### Algoritmo húngaro com distância de manhattan

método: `hung_alg_manh(nodo)`

Considere-se $n$ caixas e $m$ alvos. No jogo sokoban, $n=m$ para qualquer jogo. 

Como para cada $n_i$, só pode haver um $m_j$, e cada $n_i$ pode ir para qualquer $m_j$, as combinações deste problema formam um grafo bipartido completo $K_{n,m}$, como estudado na cadeira de Grafos e Redes.

Como o interesse é obter uma solução optima (menor custo) para este grafo, chega-se à conclusão que este é um Problema de Afetação, como estudado em Investigação Operacional.

Foi também escolhido uma forma de calcular um custo entre cada nodo caixa $n_i$ a um nodo alvo $m_j$ (neste caso baseado na distância de manhattan entre a caixa e o alvo).

Nesta heurística, foi usado o algoritmo hungáro (estudado, também, no âmbito da cadeira de Investigação Operacional) para associar uma caixa a um alvo, baseado na melhor combinação de custos (Minimizar o custo global de $K_{n,m}$).

A complexidade temporal deste algoritmo é de $O(n^3)$, que é bastante pesado, mas visto que nos puzzles dados têm todos $n ≤ 3$, vai ser possivel aplicar este algoritmo em tempo útil, como vamos testar na proximá secção.
Visto que implementar o algoritmo hungáro sai fora do âmbito da cadeira, foi utilizada a implementação [1]: https://github.com/bmc/munkres.

Todo o código necessário está disponivel no ficheiro hungarian.py e está sobre a licença Apache, permitindo o seu uso e modificação.

### Combinação das heuristicas anteriores

Foram também programados algumas funções que combinam as anteriores:

#### húngaro + distância eucl. a caixas: `hung_alg_manh_usher_to_box(nodo):`

#### húngaro + distância eucl. a alvos: `hung_alg_manh_usher_to_target(nodo):`


## Exemplos de execução

No ficheiro `sokoban.py` estão as classes principais para a execução do puzzle. Todo o código de análise execução (apresentado neste relatório) está no ficheiro `run-sokoban.py`.

In [5]:
from sokoban import *

## Análise dos algoritmos experimentados

Vão ser utilizados os puzzles entregues no enunciado, entregue no enunciado, para testar os algorimos disponíveis no ficheiro `search.py`, do repositório aima-python, disponibilizado nas aulas. Para além desses, foram construidos mais 7 puzzles para aumentar a amostra estudada.

A função (definida em `run-sokoban`) `statistics` é método  para imprimir dados da resolução de um problema Sokoban.

In [7]:
def statistics(resultado, caminho=False):
    path = resultado.path()
    solucao = resultado.solution()
    number_moves = 0
    number_pushes = 0

    for index, action in enumerate(solucao):
        accao, _ = action.split()
        if accao == 'andar':
            number_moves += 1
        else:
            number_pushes += 1

    for index, state in enumerate(path):
        if caminho:
            print(state)
    else:
        print('Número de passos:', index)

    print('Números de moves:', number_moves)
    print('Números de pushes:', number_pushes)

In [ ]:
%%timeit 
ucs_resultado = uniform_cost_search(sokoban)

In [ ]:
%%timeit
bfs_resultado = breadth_first_search(sokoban)

In [ ]:
bfs_resultado = breadth_first_search(sokoban)
statistics(bfs_resultado)

In [ ]:
ucs_resultado = uniform_cost_search(sokoban)
statistics(ucs_resultado)

In [ ]:
%%timeit
astar_resultado = astar_search(sokoban, hung_alg_manh)

In [ ]:
from hungarian import Munkres

def hung_alg_manh(nodo):
    """Algoritmo hungaro, em que o custo de cada caixa a um alvo é a distância de manhattan."""
    m = Munkres()
        
    caixas = nodo.state.caixas
    alvos = nodo.state.alvos
    custo = list()
    mhd = 0

    for index, c in enumerate(caixas):
        custo.append(list())
        for a in alvos:
            custo[index].append(abs(c[0] - a[0]) + abs(c[1] - a[1]))
    
    indexes = m.compute(custo)
    for row, column in indexes:
        value = custo[row][column]
        mhd += value
    
    return mhd

In [ ]:
astar_resultado = astar_search(sokoban, hung_alg_manh)
statistics(astar_resultado, caminho=True)